# Template 1
- 2 training runs are also in the test set
- Format a submission CSV file with data from the training set
- In fact, those runs are not in the real, hidden test set and the scoring is made on a privately rerun notebook with no internet access

In [1]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
from natsort import natsorted
import pandas as pd
import numpy as np
import zarr
# Save patches as independent files
data_dir = Path(os.environ['DATA_DIR'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
EXPERIMENTS = ['TS_5_4', 'TS_6_4', 'TS_69_2', 'TS_6_6', 'TS_73_6', 'TS_86_3',
       'TS_99_9']
OBJECT_NAMES = ['ribosome', 'virus-like-particle', 'beta-galactosidase',
         'beta-amylase', 'apo-ferritin', 'thyroglobulin']

In [3]:
particles = pd.read_csv(data_dir / "objects.csv")
particles

,experiment,particle_type,x,y,z
0,TS_5_4,ribosome,4601.271,601.066,600.934
1,TS_5_4,ribosome,4803.789,455.425,514.016
2,TS_5_4,ribosome,4715.436,825.374,802.166
3,TS_5_4,ribosome,5003.275,782.745,802.480
4,TS_5_4,ribosome,710.459,3815.845,1405.435
...,...,...,...,...,...
1264,TS_99_9,thyroglobulin,2790.000,3945.000,790.000
1265,TS_99_9,thyroglobulin,1110.308,5734.615,862.154
1266,TS_99_9,thyroglobulin,3148.098,3234.683,952.634
1267,TS_99_9,thyroglobulin,1541.778,4082.630,989.996


In [8]:
particles = pd.read_csv(data_dir / "objects.csv").reset_index(drop=False).rename(columns={'index': 'id'})
particles = particles[particles['experiment'].isin(["TS_69_2", "TS_5_4"])]
particles

,id,experiment,particle_type,x,y,z
0,0,TS_5_4,ribosome,4601.271,601.066,600.934
1,1,TS_5_4,ribosome,4803.789,455.425,514.016
2,2,TS_5_4,ribosome,4715.436,825.374,802.166
3,3,TS_5_4,ribosome,5003.275,782.745,802.480
4,4,TS_5_4,ribosome,710.459,3815.845,1405.435
...,...,...,...,...,...,...
471,471,TS_69_2,thyroglobulin,785.587,2630.000,576.201
472,472,TS_69_2,thyroglobulin,3362.941,695.882,620.588
473,473,TS_69_2,thyroglobulin,3625.942,4047.846,729.154
474,474,TS_69_2,thyroglobulin,2618.947,369.774,749.173


In [9]:
particles.to_csv(data_dir / "submission-1.csv", index=False)

In [4]:
pd.read_csv(data_dir / "sample_submission.csv")


,id,experiment,particle_type,x,y,z
0,0,TS_5_4,beta-amylase,2983.596,3154.13,764.124
1,1,TS_5_4,beta-amylase,2983.596,3154.13,764.124
2,2,TS_5_4,beta-galactosidase,2983.596,3154.13,764.124
3,3,TS_6_4,ribosome,2983.596,3154.13,764.124
4,4,TS_6_4,ribosome,2983.596,3154.13,764.124
5,5,TS_6_4,ribosome,2983.596,3154.13,764.124
6,6,TS_69_2,apo-ferritin,2983.596,3154.13,764.124
7,7,TS_69_2,beta-galactosidase,2983.596,3154.13,764.124
8,8,TS_69_2,ribosome,2983.596,3154.13,764.124
9,9,TS_69_2,virus-like-particle,2983.596,3154.13,764.124


## Determine the range of coordinates of the particles in the training set

In [10]:
particles[list("xyz")].max()

x    6229.592
y    6266.944
z    1405.435
dtype: float64

In [11]:
particles[list("xyz")].min()

x     81.893
y    116.316
z     50.000
dtype: float64